In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This script is used to estimate the number of features with strong regulatory action in gastrulation data.

In [2]:
from IPython import display
import pandas as pd
import numpy as np
import numpy as np
import numpy.ma as ma
import scipy
import scipy.stats
from scipy.stats import gaussian_kde
from scipy.stats import t

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib.offsetbox import AnchoredText

from tensorflow import keras
from sklearn.neighbors import KernelDensity

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer
Folder = '/content/drive/MyDrive/'

Load posterior latent correlations for original and negative control data.

In [3]:
#To run analysis on the accessibility/expression original or negative control data uncomment relevant lines
#with open(Folder+'SCRaPL/Synth/Results/Cells/Paper_rev/nuts_cor_1000_800.pickle', 'rb') as handle:
ng = 1
with open(Folder+'SCRaPL/Synth/Results/Cells/Paper_rev/Neg/nuts_cor_1000_800_'+str(ng)+'.pickle', 'rb') as handle:
      cor_nuts = pickle.load(handle)

In [4]:
def integral(y, x):
    dx = (x[:,-1] - x[:,0]) / (int(x.shape[1]) - 1)
    return  tf.multiply((y[:,0] + y[:,-1])/2+tf.reduce_sum(y[:,1:-1],axis=1) , dx)

def ft_detect(alpha,prob_bay):

      p_c = 1-prob_bay
      zz1 = tf.greater(p_c,alpha)
      zz2 = tf.less_equal(p_c,alpha)  

      EFDR = tf.reduce_mean(tf.gather(prob_bay,tf.where(zz1==True)))
      num_fts = tf.shape(tf.where(zz1==True))[0]
      fts_ind = tf.cast(zz1,dtype=tf.int16)

      return num_fts,EFDR,fts_ind

In [5]:
from sklearn.neighbors import KernelDensity

cor_exmp = cor_nuts.numpy()
x_d = np.linspace(-3, 3, 400)

xx= np.apply_along_axis(lambda x: KernelDensity(bandwidth=0.1, kernel='gaussian').fit(x[:,None]),0, cor_exmp)
gam = 0.115

In [6]:
p_bay  = np.zeros(1000)#np.zeros(9480)
for ii in range(p_bay.shape[0]):
    pp =  np.exp(xx[ii].score_samples(x_d[:,None]))
    p_bay[ii] =  integral(pp[None,np.abs(x_d)<2*np.math.atanh(gam)], x_d[None,np.abs(x_d)<2*np.math.atanh(gam)])
    if np.mod(ii,100)==0:
      print(ii)

0
100
200
300
400
500
600
700
800
900


In [7]:
alpha = 0.885
pp1 = 1-p_bay

zz1 = tf.greater(pp1,alpha)
zz2 = tf.less_equal(pp1,alpha)  

EFDR = tf.reduce_mean(tf.gather(p_bay,tf.where(zz1==True)))

num_features = tf.shape(tf.where(zz1==True))[0]
feature_ind = tf.cast(zz1,dtype=tf.int16)
print(EFDR)
print(num_features)

tf.Tensor(0.1069755986939622, shape=(), dtype=float64)
tf.Tensor(1, shape=(), dtype=int32)


In [8]:
alpha = np.linspace(0.0, 1.0, 201)
for ii in range(len(alpha)):
    num_features,EFDR,ft_ind = ft_detect(alpha[ii],p_bay)
    if EFDR<0.1:
          summary = tf.stack([num_features.numpy(),EFDR.numpy(),alpha[ii]]).numpy()
          print(summary)
          break
if np.isnan(EFDR.numpy()):
        print("No significant feature detected")

No significant feature detected


In [9]:
#0 --> 115/880 --> 715
#1 --> 115/895 --> 0
#2 --> 115/835 --> 1
#3 --> 115/790 --> 4
#4 --> 115/870 --> 0
#5 --> 115/885 --> 0

In [10]:
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/EFDR_neg'+str(neg)+".csv", EFDR.numpy()[np.newaxis,np.newaxis], delimiter=",")
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/feature_ind_neg'+str(neg)+".csv", feature_ind.numpy() , delimiter=",")
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/p_bay_neg'+str(neg)+".csv", p_bay , delimiter=",")